### Interaction API

In this section, we will continue investigating the use of an API published by the National Library of Medicine.  The techniques will be very similar to the spell checker example in the APIRequest notebook, though the data is more elaborate and more challenging to parse.

We will be querying drug interactions through teh InteractionAPI.  This api accepts an rxcui for a particular drug and provides information about the known interactions with other medications.  

To read more about this API, you can follow the link at

https://rxnav.nlm.nih.gov/InteractionAPIs.html

For an overview of the metadata related to this api, you can use the appendix at

https://www.nlm.nih.gov/research/umls/rxnorm/docs/2015/appendix5.html

We start by loading the required libraries, making a connection to a URI, and converting the JSON response to a python dictionary

In [29]:
import requests

url = 'https://rxnav.nlm.nih.gov/REST/interaction/interaction.json?rxcui=153165'
resp = requests.get(url)
data = resp.json()

this cell is commented out because of the amount of data.  To follow along, you can uncomment and run to look at the raw JSON from the URI.  Alternatively, just take a look at the URI directly in your brower.  You may want to install a plug-in to make viewing easier.

In [30]:
# data

In [31]:
type(data)

dict

This data structure is more elaborate and nested than the spell checker, combining lists and dictionaries.  

Suppose you wanted to use this API to extract a list of drugs with a known interaction?  Do do this, you'll need to parse the JSON document to get specifically at the data fields you need.  There are various techniques to do this, so you often don't need to write this by hand.  However, there will be times when you need to apply very specific logic to a JSON tree, so we will investigate how to do this by directly accessing elements of a tree with Python.  

To directly access the names of the drugs with interactions, we will need to follow

interactionTypeGroup -> interactionType -> interactionPair -> minConceptItem.

Some of these are nested dictionaries, and some are lists, so we may need to use different syntax to access them.

To get the top level keys for this dictionary, you can use the python keys syntax.  Remember, a JSON document is very similar to a dictionary, and once loaded, you can use all the dict methods available in Python.

In [32]:
data.keys()

dict_keys(['nlmDisclaimer', 'userInput', 'interactionTypeGroup'])

Becase we are dealing with **keys**, we can use dictionary syntax.  

In [33]:
interactionTypeGroup = data['interactionTypeGroup']

In [34]:
type(interactionTypeGroup)

list

We have a key that maps to a list.  Let's check the length.

In [35]:
len(interactionTypeGroup)

1

There's only one item in this list, but we can't assume that will always be the case.  If you want to parse a JSON tree, or a nested dictionary, make sure you iterate over every item in a list, even if in every case so far, you've only seen a single value.  

We're dealing with a list here, so instead of using a lookup value in a dictionary (which is not ordered), we can access by positional index for a list, which is ordered.

In [36]:
interactionTypeGroup = interactionTypeGroup[0]

In [37]:
type(interactionTypeGroup)

dict

The top level dictionary contained a list.  This list, in turn, contains dictionaries.  

In [38]:
interactionTypeGroup.keys()

dict_keys(['sourceDisclaimer', 'sourceName', 'interactionType'])

Using this technique, let's continue to descend this tree until we get to the data we need.

In [39]:
interactionType = interactionTypeGroup['interactionType']

In [40]:
type(interactionType)

list

In [41]:
len(interactionType)

1

In [42]:
interactionType = interactionType[0]

In [43]:
interactionType.keys()

dict_keys(['minConceptItem', 'interactionPair', 'comment'])

In [44]:
interactionPair = interactionType['interactionPair']

In [45]:
#interactionPair

In [46]:
len(interactionPair)

368

To avoid ambiguity between terms, let's pluralize this to remember we're dealing with a list.  The naming makes no different to Python, of course, but we (and others) have to read this code!

In [47]:
interactionPairs = interactionPair

In [48]:
interactionPair[0]

{'description': 'The serum concentration of Atorvastatin can be increased when it is combined with Lepirudin.',
 'interactionConcept': [{'minConceptItem': {'name': 'atorvastatin',
    'rxcui': '83367',
    'tty': 'IN'},
   'sourceConceptItem': {'id': 'DB01076',
    'name': 'Atorvastatin',
    'url': 'http://www.drugbank.ca/drugs/DB01076#interactions'}},
  {'minConceptItem': {'name': 'lepirudin', 'rxcui': '237057', 'tty': 'IN'},
   'sourceConceptItem': {'id': 'DB00001',
    'name': 'Lepirudin',
    'url': 'http://www.drugbank.ca/drugs/DB00001#interactions'}}],
 'severity': 'N/A'}

In [49]:
type(interactionPair[0])

dict

In [50]:
interactionPair[0].keys()

dict_keys(['interactionConcept', 'severity', 'description'])

We don't have to assign an intermediary variable if we don't need one - we can grab the key value from the first element of the array.  

In [51]:
interactionConcept = interactionPair[0]['interactionConcept']

In [52]:
interactionConcept

[{'minConceptItem': {'name': 'atorvastatin', 'rxcui': '83367', 'tty': 'IN'},
  'sourceConceptItem': {'id': 'DB01076',
   'name': 'Atorvastatin',
   'url': 'http://www.drugbank.ca/drugs/DB01076#interactions'}},
 {'minConceptItem': {'name': 'lepirudin', 'rxcui': '237057', 'tty': 'IN'},
  'sourceConceptItem': {'id': 'DB00001',
   'name': 'Lepirudin',
   'url': 'http://www.drugbank.ca/drugs/DB00001#interactions'}}]

In [53]:
len(interactionConcept)

2

Look back at your original tree - the first element of the list is the name of the drug we're researching.  The second element is the name of the drug with an interaction.  To build our list, we'll grab that one.  

In [54]:
minConceptItem = interactionConcept[1]['minConceptItem']

In [35]:
minConceptItem

{'name': 'lepirudin', 'rxcui': '237057', 'tty': 'IN'}

Finally, we arrive at a the data we need!

In [55]:
name = minConceptItem['name']

In [37]:
name

'lepirudin'

### Exercise

We obtained the name for a single interaction in this section.  How would you parse this tree to obtain a list of all interactions from this web service?  

There are always multiple ways to do things in a programming language.  In the next section, InteractionList.ipynb, we'll review one way to do this.  But before you take a look, give this a try and see how your solution compares.  